In [1]:
%load_ext lab_black

In [1]:
import pandas as pd
from nltk import tokenize
import boto3
import botocore
from datetime import datetime
import matplotlib.pylab as plt
import yfinance as yf
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from numba import jit, prange

In [2]:
# Load appropriate packages for sentiment analysis
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.data import Sentence
from segtok.segmenter import split_single
import pandas as pd
import re


classifier = TextClassifier.load("en-sentiment")

2021-03-10 10:31:53,031 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_3.1.pt


In [7]:
nvidia_cleaned_df = pd.read_csv("../00_data/Cleaned_Data_10MAR.gz", compression="gzip")

In [8]:
nvidia_cleaned_df["VideoTitle"] = nvidia_cleaned_df["VideoTitle"].astype("str")
nvidia_cleaned_df["datePub"] = pd.to_datetime(
    nvidia_cleaned_df["datePub"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_cleaned_df["searchedDate"] = pd.to_datetime(
    nvidia_cleaned_df["searchedDate"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_cleaned_df["channelTitle"] = nvidia_cleaned_df["channelTitle"].astype("str")
nvidia_cleaned_df["viewCount"] = nvidia_cleaned_df["viewCount"].astype("int")
nvidia_cleaned_df["likeCount"] = nvidia_cleaned_df["likeCount"].astype("int")
nvidia_cleaned_df["dislikeCount"] = nvidia_cleaned_df["dislikeCount"].astype("int")
nvidia_cleaned_df["captionString"] = nvidia_cleaned_df["captionString"].astype("str")

In [9]:
nvidia_cleaned_df.dtypes

Unnamed: 0                     int64
Index                          int64
videoID                       object
datePub          datetime64[ns, UTC]
searchedDate     datetime64[ns, UTC]
VideoTitle                    object
channelTitle                  object
viewCount                      int32
likeCount                      int32
dislikeCount                   int32
captionString                 object
dtype: object

In [10]:
nvidia_cleaned_df.head()

,Unnamed: 0,Index,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,captionString
0,0,0,y4fb_R5Ogw0,2020-10-01 05:16:02+00:00,2020-10-02 01:45:21.553469+00:00,"2x NVIDIA RTX 3090 SLI Benchmarks: 500FPS, 700...",Gamers Nexus,367472,15701,260,so this setup that you're seeing is what we ha...
1,1,1,ALEXVtnNEwA,2020-09-01 17:15:57+00:00,2020-10-02 01:45:22.351801+00:00,NVIDIA GeForce Special Event Livestream,IGN,543115,16860,411,b welcome to my kitchen i hope all of you are ...
2,2,2,E98hC9e__Xs,2020-09-01 16:42:07+00:00,2020-10-02 01:45:23.252012+00:00,NVIDIA GeForce RTX 30 Series | Official Launch...,NVIDIA GeForce,1484075,54741,1135,Welcome to my kitchen. I hope all of you are...
3,3,3,nMns8t4OhI0,2020-09-30 23:00:06+00:00,2020-10-02 01:45:23.922084+00:00,NVIDIA Responds to RTX 3080 Crashes...,JayzTwoCents,561238,27766,885,nan
4,4,4,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2020-10-02 01:45:24.592244+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3391565,141852,3961,- The RTX 3080 is the most powerful GPU we've ...


In [11]:
nvidia_cleaned_df["Date"] = pd.to_datetime(
    nvidia_cleaned_df["searchedDate"], format="%Y-%m-%d", utc=True
).dt.date

Vader Sentiment

In [13]:
# load vader sentiment for comparing the uncleaned strings to the modified ones
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [14]:
nvidia_cleaned_df.columns

Index(['Unnamed: 0', 'Index', 'videoID', 'datePub', 'searchedDate',
       'VideoTitle', 'channelTitle', 'viewCount', 'likeCount', 'dislikeCount',
       'captionString', 'Date'],
      dtype='object')

In [15]:
print(f'Title to score = {nvidia_cleaned_df["captionString"][5]}')
score = sid.polarity_scores(str(nvidia_cleaned_df["captionString"][5]))
print(score)

Title to score = (clapboard sounds) - Did you guys just seethe same thing I did? RTX 3070 is gonna outperform a 2080Ti at less than half the price, 499. Is Nvidia insane. I mean, even with all of the leaks indicating that Ampere wasgonna be super powerful, I was in no way ready for this, so okay. (sighs) Deep breath, there's alot to get through here, so let's get started. After an F in the chat forany recent 2080Ti buyers. (giggles) Ouch, like dang. Oh and also our sponsor. Ridge Wallet wants to redefine the wallet with its compact frameand RFID blocking plates. Check out how they can helpkeep your wallet bulge down and use our offer code Linus to save 10% and getfree worldwide shipping. (upbeat music) Now, obviously I'm notready to hand Nvidia their crown until we seethe new cards for ourselves. But based on what they've shown, we are in for one heck of a ride. They announced well a lot andwe're gonna get to all of it, but the big one is, Ampere is real and Ampere is an absolute monst

In [16]:
# utilize regular expressions to split large stings into sentences that are centered around the key words
import re

text = " allet wants to redefine the wallet with its compact frameand RFID blocking plates. Check out how they can helpkeep your wallet bulge down and use our offer code Linus to save 10% and getfree worldwide shipping. (upbeat music) Now, obviously I'm notready to hand Nvidia their crown until we seethe new cards for ourselves. But based on what they've shown, we are in for one heck of a ride. They announced well a lot andwe're gonna get to all of it, but the big one is, Ampere is real and Ampere is an absolute monster. As it turns out, the recentleaks were accurate in many if not most ways."


def search(target, text, context=6):
    # It's easier to use re.findall to split the string,
    # as we get rid of the punctuation
    words = re.findall(r"\w+", text)

    matches = (i for (i, w) in enumerate(words) if w.lower() == target)
    for index in matches:
        if index < context // 2:
            yield words[0 : context + 1]
        elif index > len(words) - context // 2 - 1:
            yield words[-(context + 1) :]
        else:
            yield words[index - context // 2 : index + context // 2 + 1]


print(list(search("nvidia", text, 10)))

[['I', 'm', 'notready', 'to', 'hand', 'Nvidia', 'their', 'crown', 'until', 'we', 'seethe']]


In [17]:
search("nvidia", text, 15)

<generator object search at 0x000001DAA606B200>

In [18]:
test_list_to_str = list(search("nvidia", text, 30))
test_list_to_str

[['save',
  '10',
  'and',
  'getfree',
  'worldwide',
  'shipping',
  'upbeat',
  'music',
  'Now',
  'obviously',
  'I',
  'm',
  'notready',
  'to',
  'hand',
  'Nvidia',
  'their',
  'crown',
  'until',
  'we',
  'seethe',
  'new',
  'cards',
  'for',
  'ourselves',
  'But',
  'based',
  'on',
  'what',
  'they',
  've']]

In [19]:
new_str = " ".join(test_list_to_str[0])

In [20]:
score = sid.polarity_scores(new_str)
print(score)

{'neg': 0.0, 'neu': 0.808, 'pos': 0.192, 'compound': 0.7506}


### Flair Sentiment

In [21]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load("en-sentiment")
sentence = Sentence("Happy to be here")
classifier.predict(sentence)
# print sentence with predicted labels
print("Sentence above is: ", sentence.labels)

2021-04-06 20:18:24,105 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_4.pt
Sentence above is:  [POSITIVE (0.998)]


In [22]:
sentence.labels

[POSITIVE (0.998)]

In [23]:
sentence = Sentence("Happy to be here")
classifier.predict(sentence)

### Nvidia Breakout Sentiment

In [82]:
# Load appropriate packages for sentiment analysis
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.data import Sentence
from segtok.segmenter import split_single
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
classifier = TextClassifier.load("en-sentiment")

2021-04-06 20:38:10,366 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_4.pt


In [3]:
# utilize regular expressions to split large stings into sentences that are centered around the key words
import re

text = (
    "Education shall be directed to the full development of the human personality "
    "and to the strengthening of respect for human rights and fundamental freedoms."
)


def search(target, text, context=6):
    # It's easier to use re.findall to split the string,
    # as we get rid of the punctuation
    words = re.findall(r"\w+", text)

    matches = (i for (i, w) in enumerate(words) if w.lower() == target)
    for index in matches:
        if index < context // 2:
            yield words[0 : context + 1]
        elif index > len(words) - context // 2 - 1:
            yield words[-(context + 1) :]
        else:
            yield words[index - context // 2 : index + context // 2 + 1]


print(list(search("the", text, 3)))
# [['be', 'directed', 'to', 'the', 'full', 'development', 'of'],
#  ['full', 'development', 'of', 'the', 'human', 'personality', 'and'],
#  ['personality', 'and', 'to', 'the', 'strengthening', 'of', 'respect']]

[['to', 'the', 'full'], ['of', 'the', 'human'], ['to', 'the', 'strengthening']]


In [134]:
def flair_predict(sentence):
    """ Predict the sentiment of a sentence """
    if sentence == "":
        return 0
    text = Sentence(sentence)
    # stacked_embeddings.embed(text)
    classifier.predict(text)
    value = text.labels[0].to_dict()["value"]
    if value == "POSITIVE":
        result = text.to_dict()["labels"][0]["confidence"]
    else:
        result = -(text.to_dict()["labels"][0]["confidence"])
    return round(result, 3)

def vader_predict(sentence):
    """ Predict the sentiment of a sentence """
    if sentence == "":
        return 0
    # stacked_embeddings.embed(text)
    else: 
        score = sid.polarity_scores(" ".join(sentence))
        result = score['pos']
        return round(result, 3)

In [135]:
def get_scores(sentences, flairPredict = True):
    """ Call predict on every sentence of a text """
    results = []

    if flairPredict == True:
        for i in range(0, len(sentences)):
            results.append(flair_predict(sentences[i]))
        return results
    else:
        for i in range(0, len(sentences)):
            results.append(vader_predict(sentences[i]))
        return results

def get_scores_single_cap(sentences, flairPredict = True):
    """ Call predict on single caption string """
    if flairPredict == True:
        results = []
        results.append(flair_predict(sentences))
        return results
    else:
        results = []
        results.append(vader_predict(sentences))
        return results

def get_sum(scores):
    result = round(sum(scores), 3)
    return result


def get_avg(scores):
    result = round((sum(scores) / len(scores)), 3)
    return result

In [142]:
test_1 = nvidia_cleaned_df["captionString"][4]
nvidia_cleaned_df["VideoTitle"][4]

'Nvidia, you PROMISED! - RTX 3080 Review'

In [143]:
test_1

"- The RTX 3080 is the most powerful GPU we've ever seen at this price point. But, we've said that before. And in fact we say it every time there's a new graphics card generation. So what makes this one so darn special? I mean, the hypesurrounding it was so huge that even we got caught up in it. I mean, look how excited this guy is, it's infectious, but as weall know, hype disappoints and absolute hype disappoints absolutely. Unlike our sponsor, GlassWire. GlassWire is the tool that shows you which apps are slowingdown your connection in real time and it'sused by security pros to monitor for malware,block bandwidth wasters and detect suspicious activity. Get 25% off today at the link below. (upbeat music) Do you hear that? It's rattling. Weird. While you read over our test platform, let me get this off of my chest. NVIDIA, man you guysreally went out of your way to set yourselves up here, didn't you? Like I don't get it. Spoiler alert, you're shippingthe fastest graphics card on the ma

In [144]:
#list_test
print(list_test[3])
print("Flair Score")
print(flair_predict(list_test[3]))
print("Vader Score")
print(vader_predict(list_test[3]))

['latency', 'analyzer', 'just', 'connect', 'your', 'mouse', 'the', 'nvidia', '360', 'hertz', 'g', 'sync', 'esports', 'displays', 'are']
Flair Score
-0.646
Vader Score
0.0


In [145]:
list_test = list(search("nvidia", test_1, 15))

In [148]:
list_test

[['me',
  'get',
  'this',
  'off',
  'of',
  'my',
  'chest',
  'NVIDIA',
  'man',
  'you',
  'guysreally',
  'went',
  'out',
  'of',
  'your'],
 ['reasonI',
  'm',
  'disappointed',
  'at',
  'all',
  'is',
  'because',
  'NVIDIA',
  'told',
  'me',
  'to',
  'expect',
  'double',
  'the',
  'performance'],
 ['XT',
  'so',
  'we',
  'can',
  'test',
  'out',
  'DLSS',
  'NVIDIA',
  's',
  'deep',
  'learningup',
  'sampling',
  'technology',
  'And',
  'ooh'],
 ['then',
  'whileit',
  'may',
  'not',
  'be',
  'exactly',
  'what',
  'NVIDIA',
  'promised',
  'us',
  'the3080',
  'is',
  'still',
  'a',
  'gaming'],
 ['gives',
  'us',
  'a',
  'hint',
  'as',
  'to',
  'where',
  'NVIDIA',
  'may',
  'have',
  'been',
  'focusing',
  'when',
  'they',
  'came'],
 ['let',
  's',
  'look',
  'at',
  'power',
  'draw',
  'Using',
  'NVIDIA',
  's',
  'PCAT',
  'tool',
  'we',
  'captured',
  'this',
  'datawhile'],
 ['captured',
  'this',
  'datawhile',
  'running',
  'SPECviewperf',
  

In [146]:
num = 0
print("Flair")
for string in list_test:
    int_score = get_scores_single_cap(string, flairPredict = True)
    print(num, int_score, string)
    num += 1
    
print("BREAK BREAK BREAK")

print("Vader")
num =0
for string in list_test:
    int_score = get_scores_single_cap(string, flairPredict = False)
    print(num, int_score, string)
    num += 1

Flair
0 [-0.945] ['me', 'get', 'this', 'off', 'of', 'my', 'chest', 'NVIDIA', 'man', 'you', 'guysreally', 'went', 'out', 'of', 'your']
1 [-1.0] ['reasonI', 'm', 'disappointed', 'at', 'all', 'is', 'because', 'NVIDIA', 'told', 'me', 'to', 'expect', 'double', 'the', 'performance']
2 [0.981] ['XT', 'so', 'we', 'can', 'test', 'out', 'DLSS', 'NVIDIA', 's', 'deep', 'learningup', 'sampling', 'technology', 'And', 'ooh']
3 [-0.985] ['then', 'whileit', 'may', 'not', 'be', 'exactly', 'what', 'NVIDIA', 'promised', 'us', 'the3080', 'is', 'still', 'a', 'gaming']
4 [0.988] ['gives', 'us', 'a', 'hint', 'as', 'to', 'where', 'NVIDIA', 'may', 'have', 'been', 'focusing', 'when', 'they', 'came']
5 [0.75] ['let', 's', 'look', 'at', 'power', 'draw', 'Using', 'NVIDIA', 's', 'PCAT', 'tool', 'we', 'captured', 'this', 'datawhile']
6 [0.999] ['captured', 'this', 'datawhile', 'running', 'SPECviewperf', 'on', 'our', 'NVIDIA', 'cards', 'here', 'and', 'we', 'can', 'see', 'that']
7 [1.0] ['the', 'trend', 'continues', 'i

In [147]:
print("Flair")
print("Average flair score", get_avg(get_scores(list_test)))
print("Vader")
print("Average vader score", get_avg(get_scores(list_test, flairPredict = False)))

Flair
Average flair score 0.211
Vader
Average vader score 0.093


In [79]:
get_scores(list_test)

[0.999,
 -0.968,
 -0.638,
 -0.97,
 0.997,
 1.0,
 0.993,
 0.978,
 1.0,
 -0.913,
 0.997,
 0.999,
 0.847,
 -0.982,
 0.952,
 0.996,
 1.0,
 0.994,
 0.874,
 0.993,
 -1.0,
 0.999,
 1.0,
 0.977,
 0.983,
 0.982,
 1.0,
 0.999,
 1.0,
 0.996,
 1.0,
 0.997,
 1.0,
 0.999]